this is the notebook where I will augment my data extract my Features,and decide what architecture to use and train my model

In [1]:

import numpy as np
import pandas as pd
import torch
import torchaudio
import warnings
import os
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
from torchsummary import summary
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

import librosa
from torchaudio.functional import preemphasis

import torch.nn.functional as F
import random
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [2]:
import os
os.getcwd()

'c:\\CNN Project'

In [3]:
df=pd.read_csv("dataset2.csv")
df.head()

,path,chapter,reciter
0,abd_albassit_abd_samad_1_rir.wav,surah_al_dhoha,abd_albassit_abd_samad
1,abd_albassit_abd_samad_1_org.wav,surah_al_dhoha,abd_albassit_abd_samad
2,abd_albassit_abd_samad_2_bg.wav,surah_al_dhoha,abd_albassit_abd_samad
3,abd_albassit_abd_samad_2_org.wav,surah_al_dhoha,abd_albassit_abd_samad
4,abd_albassit_abd_samad_3_bg.wav,surah_al_dhoha,abd_albassit_abd_samad


In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

#Data Augmentation

my possible options:-Noise Injection/-Reverberation /- Background Noise Simulation/-Volume Perturbation/-Clipping/-Equalization (EQ) Changes

##Loading Data Done


In [3]:
class OptimizedAudioDataset(Dataset):
    def __init__(self, csv_file, audio_dir, transform=None, max_length=22050*5,is_test=False):#max length change 
        self.data = pd.read_csv(csv_file)
        self.audio_dir = audio_dir
        self.transform = transform
        self.max_length = max_length
        if not is_test:
            self.label_encoder = LabelEncoder()
            self.data['reciter'] = self.label_encoder.fit_transform(self.data['reciter'])
        self.is_test = is_test

        # Pre-load all audio files
        self.audio_data = self.preload_audio(self.max_length)

    def preload_audio(self,max_length):
        audio_data = {}
        for _, row in tqdm(self.data.iterrows(), total=len(self.data), desc="Preloading audio"):
            audio_path = os.path.join(self.audio_dir, row['path'])
            waveform, sample_rate = torchaudio.load(audio_path)
            if sample_rate != 22050:
                # Resample the audio
                resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=22050)
                waveform = resampler(waveform)
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            # if waveform.shape[1] < self.max_length:
            #     padding = torch.zeros(1, self.max_length - waveform.shape[1])
            #     waveform = torch.cat([waveform, padding], dim=1)
            # else:
            #     waveform = waveform[:, :self.max_length]
            if waveform.shape[-1] < max_length:
                padding = torch.zeros(max_length - waveform.shape[-1])
                padded_audio = torch.cat((waveform, padding), dim=-1)
                audio_data[row['path']] = padded_audio
            elif waveform.shape[-1] > max_length:
                truncated_audio = waveform[:, :max_length]
                audio_data[row['path']] = truncated_audio
            else:
                audio_data[row['path']] = waveform
        return audio_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        audio_filepath = self.data.iloc[idx]['path']
        waveform = self.audio_data[audio_filepath]

        if self.transform:
            waveform = self.transform(waveform)

        
        if self.is_test:
            return waveform
        else:
            label = self.data.iloc[idx]['reciter']
            return waveform, label
       

def optimized_audio_transform(waveform, sample_rate=22050, n_mels=128, n_fft=400, hop_length=120):
    # Apply pre-emphasis filter
    waveform = torchaudio.functional.preemphasis(waveform)

    # Compute mel spectrogram
    lfcc = torchaudio.transforms.LFCC(
    sample_rate=sample_rate,
    n_filter=n_mels,
    n_lfcc=64,                      # Number of LFCC coefficients to extract
    speckwargs={"n_fft": 400, "hop_length": 120, "center": False},         # Additional arguments for spectrogram computation
)(waveform)


    return lfcc.squeeze(0).transpose(0, 1)  # Reshape to (time_steps, n_lfcc)

def custom_collate_fn(batch):
    if isinstance(batch[0], tuple):
        waveforms, labels = zip(*batch)
        waveforms = torch.stack(waveforms)
        return waveforms, torch.tensor(labels)
    else:
        return torch.stack(batch)

In [7]:


train_dataset = OptimizedAudioDataset(csv_file="dataset2.csv",
                                      audio_dir="Dataset",
                                      transform=optimized_audio_transform)

train_data, val_data = train_test_split(train_dataset, test_size=0.1, random_state=42,
                                        stratify=train_dataset.data['reciter'])

train_loader = DataLoader(train_data, batch_size=100, shuffle=True,
                          collate_fn=custom_collate_fn, num_workers=0)

val_loader = DataLoader(val_data, batch_size=100, shuffle=False,
                        collate_fn=custom_collate_fn, num_workers=0)

# test_dataset = OptimizedAudioDataset(csv_file="/content/Test_1 (1).csv",
#                                      audio_dir="/content/drive/MyDrive/Cleaned_TechCabal_01",
#                                      transform=optimized_audio_transform,
#                                      is_test=True)

# test_loader = DataLoader(test_dataset, batch_size=70, shuffle=False,
#                          collate_fn=custom_collate_fn, num_workers=4)

Preloading audio: 100%|██████████| 6650/6650 [00:19<00:00, 348.64it/s]


In [ ]:
t=[]
for i in range (1000):
    data,label=train_data[i]
    t.append(data.shape[0])

In [7]:
from collections import Counter

Counter(t)

Counter({690: 1000})

##Model

In [8]:

class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation block for channel-wise attention
    """
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class TCSConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(TCSConv, self).__init__()
        self.depthwise_conv = nn.Conv1d(in_channels, in_channels, kernel_size, groups=in_channels, padding='same')
        self.pointwise_conv = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.se = SEBlock(out_channels)

    def forward(self, x):

        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        x = self.se(x)
        return x

class SubBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(SubBlock, self).__init__()
        self.tcs_conv = TCSConv(in_channels, out_channels, kernel_size)
        self.bnorm = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(p=0.1)
        self.activation = nn.ReLU()

    def forward(self, x):

        residual = x
        x = self.tcs_conv(x)
        x = self.bnorm(x)
        x = self.activation(x)
        x = self.dropout(x)
        # Add residual connection if input and output channels match
        if x.shape == residual.shape:
            x += residual
        return x

class MainBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, R=1):
        super(MainBlock, self).__init__()
        self.residual_pointwise = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.residual_batchnorm = nn.BatchNorm1d(out_channels)

        self.sub_blocks = nn.ModuleList([
            SubBlock(in_channels if i == 0 else out_channels, out_channels, kernel_size)
            for i in range(R)
        ])

    def forward(self, x):
        residual = self.residual_batchnorm(self.residual_pointwise(x))
        for layer in self.sub_blocks:
            x = layer(x)
        return x + residual  # Add the residual connection at the end of the MainBlock

class MatchboxNet(nn.Module):
    def __init__(self, B, R, C, kernel_sizes=None, NUM_CLASSES=30):
        super(MatchboxNet, self).__init__()
        if not kernel_sizes:
            kernel_sizes = [k*2+11 for k in range(1, B+1)]

        self.prologue = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=11, stride=2),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )

        self.blocks = nn.ModuleList([
            MainBlock(128 if i == 0 else C, C, kernel_size=kernel_sizes[i], R=R)
            for i in range(B)
        ])

        self.epilogue = nn.Sequential(
            nn.Conv1d(C, 128, kernel_size=29, dilation=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, NUM_CLASSES, kernel_size=1),
            nn.AdaptiveAvgPool1d(1)
        )

    def forward(self, x):
        x = self.prologue(x)
        for block in self.blocks:
            x = block(x)
        x = self.epilogue(x).squeeze(2)
        return F.log_softmax(x, dim=1)

In [10]:
model = MatchboxNet(B=3, R=2, C=64, NUM_CLASSES=17)
summary(model, input_size=(64, 746))  # Adjust input_size to     match (batch_size, input_dim),

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 128, 368]          90,240
       BatchNorm1d-2             [-1, 128, 368]             256
              ReLU-3             [-1, 128, 368]               0
            Conv1d-4              [-1, 64, 368]           8,256
       BatchNorm1d-5              [-1, 64, 368]             128
            Conv1d-6             [-1, 128, 368]           1,792
            Conv1d-7              [-1, 64, 368]           8,256
 AdaptiveAvgPool1d-8                [-1, 64, 1]               0
            Linear-9                    [-1, 4]             256
             ReLU-10                    [-1, 4]               0
           Linear-11                   [-1, 64]             256
          Sigmoid-12                   [-1, 64]               0
          SEBlock-13              [-1, 64, 368]               0
          TCSConv-14              [-1, 

Train Model 

In [11]:

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=5, grad_clip=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using..",device)
    model.to(device)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience, verbose=True)
   # scheduler = OneCycleLR(
    #optimizer,
  #  max_lr=0.01,  # Maximum learning rate
   # steps_per_epoch=steps_per_epoch,
  #  epochs=num_epochs,
   # pct_start=0.3,  # Percentage of the cycle to increase learning rate
   # anneal_strategy='cos',  # Annealing strategy ('linear' or 'cos')
   # div_factor=25.0,  # Initial learning rate = max_lr / div_factor
   # final_div_factor=1e4  # Minimum learning rate = initial_lr / final_div_factor
#)

    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = inputs.permute(0, 2, 1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient clipping
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                inputs = inputs.permute(0, 2, 1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print("--------------------")

        # Adjust learning rate based on validation loss
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'Matchboxnet_bestModel/MatchboxNet_best_model1.pt')

    print("Training completed.")

num_classes = len(train_dataset.label_encoder.classes_)
model = MatchboxNet(B=3, R=2, C=64, NUM_CLASSES=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)

Using.. cpu
Epoch [1/100] Train Loss: 1.0784, Train Acc: 0.7890 Val Loss: 0.5340, Val Acc: 0.8647
--------------------
Epoch [2/100] Train Loss: 0.2692, Train Acc: 0.9519 Val Loss: 0.3510, Val Acc: 0.8977
--------------------
Epoch [3/100] Train Loss: 0.1735, Train Acc: 0.9586 Val Loss: 0.1670, Val Acc: 0.9639
--------------------
Epoch [4/100] Train Loss: 0.1270, Train Acc: 0.9673 Val Loss: 0.9925, Val Acc: 0.7940
--------------------
Epoch [5/100] Train Loss: 0.1163, Train Acc: 0.9669 Val Loss: 0.2569, Val Acc: 0.9263
--------------------
Epoch [6/100] Train Loss: 0.1067, Train Acc: 0.9711 Val Loss: 0.1921, Val Acc: 0.9429
--------------------
Epoch [7/100] Train Loss: 0.0865, Train Acc: 0.9744 Val Loss: 0.1967, Val Acc: 0.9414
--------------------
Epoch [8/100] Train Loss: 0.0868, Train Acc: 0.9758 Val Loss: 0.3336, Val Acc: 0.9188
--------------------
Epoch [9/100] Train Loss: 0.0687, Train Acc: 0.9803 Val Loss: 0.1445, Val Acc: 0.9549
--------------------
Epoch [10/100] Train Loss

In [12]:
model_inference = MatchboxNet(B=3, R=2, C=64, NUM_CLASSES=17)
model_inference.load_state_dict(torch.load('Matchboxnet_bestModel/MatchboxNet_best_model1.pt'))

<All keys matched successfully>

Inference

In [13]:
test_dataset = OptimizedAudioDataset(csv_file="test.csv",
                                      audio_dir="test/Test",
                                      transform=optimized_audio_transform,
                                      is_test=True)

test_loader = DataLoader(test_dataset, batch_size=70, shuffle=False,
                          collate_fn=custom_collate_fn, num_workers=0)
                        
def predict_test_data(model, test_loader):
    device = "cpu"
    model.to(device)
    print("MODEL is using CPU for Inferencing....")
    model.eval()
    predictions = []

    with torch.inference_mode():
        for inputs in tqdm(test_loader, desc="Inferencing"):
            inputs = inputs.to(device)
            inputs = inputs.permute(0, 2, 1)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

test_predictions = predict_test_data(model_inference, test_loader)

Preloading audio: 100%|██████████| 7/7 [00:00<00:00, 77.96it/s]


MODEL is using CPU for Inferencing....


Inferencing: 100%|██████████| 1/1 [00:00<00:00,  6.71it/s]


In [14]:
test=pd.read_csv("test.csv")
test.head(8)

,path,label
0,C:\CNN Project\test\Test\Ahmed_ali_ajmi.wav,Ahmed_ali_ajmi.m4a
1,C:\CNN Project\test\Test\Alimahmoud.wav,Alimahmoud.m4a
2,C:\CNN Project\test\Test\Islem_sob7i.wav,Islem_sob7i.m4a
3,C:\CNN Project\test\Test\Yaser_aldousari.wav,Yaser_aldousari.m4a
4,C:\CNN Project\test\Test\Youssef_aidours.wav,Youssef_aidours.m4a
5,C:\CNN Project\test\Test\phone_audio.wav,yasser
6,C:\CNN Project\test\Test\surah_al_mulk-yasser_...,yasser


In [15]:
train_dataset.label_encoder.inverse_transform(test_predictions)


array(['ahmed_alijmi', 'maher_alm3i9li', 'ahmed_alijmi',
       'abd_albassit_abd_samad', 'maher_alm3i9li', 'yasser_aldossari',
       'yasser_aldossari'], dtype=object)